In [3]:
! lscpu

Arquitectura:                            x86_64
  modo(s) de operación de las CPUs:      32-bit, 64-bit
  Address sizes:                         39 bits physical, 48 bits virtual
  Orden de los bytes:                    Little Endian
CPU(s):                                  16
  Lista de la(s) CPU(s) en línea:        0-15
ID de fabricante:                        GenuineIntel
  Nombre del modelo:                     Intel(R) Core(TM) i7-10700 CPU @ 2.90GH
                                         z
    Familia de CPU:                      6
    Modelo:                              165
    Hilo(s) de procesamiento por núcleo: 2
    Núcleo(s) por «socket»:              8
    «Socket(s)»                          1
    Revisión:                            5
    CPU MHz máx.:                        4800,0000
    CPU MHz mín.:                        800,0000
    BogoMIPS:                            5799.77
    Indicadores:                         fpu vme de pse tsc msr pae mce cx8 apic
       

Frecuencia del CPU: 2800 MHz

In [138]:
%%file suma_cubos.c

#include <stdio.h>
/*el puntero ingresado es para dirección de un entero y la función devuelve la suma de num enteros*/
int suma_cubo(int* val, int N){
    int suma_c = 0;
    for(int i=0;i<N;i++){
        suma_c = suma_c + val[i]*val[i]*val[i];
    }
    return suma_c;
}


Overwriting suma_cubos.c


In [141]:
! gcc -Os -c suma_cubos.c -o suma_cubos.o

In [142]:
! objdump -M intel -j .text -D suma_cubos.o


suma_cubos.o:     formato del fichero elf64-x86-64


Desensamblado de la sección .text:

0000000000000000 <suma_cubo>:
   0:	f3 0f 1e fa          	endbr64 
   4:	31 c0                	xor    eax,eax
   6:	45 31 c0             	xor    r8d,r8d
   9:	39 c6                	cmp    esi,eax
   b:	7e 13                	jle    20 <suma_cubo+0x20>
   d:	8b 0c 87             	mov    ecx,DWORD PTR [rdi+rax*4]
  10:	48 ff c0             	inc    rax
  13:	89 ca                	mov    edx,ecx
  15:	0f af d1             	imul   edx,ecx
  18:	0f af d1             	imul   edx,ecx
  1b:	41 01 d0             	add    r8d,edx
  1e:	eb e9                	jmp    9 <suma_cubo+0x9>
  20:	44 89 c0             	mov    eax,r8d
  23:	c3                   	ret    


In [164]:
%%file main.c

#include <stdio.h>
#include <stdlib.h>
#include <x86intrin.h>
#include <time.h>
/*la cantidad de instrucciones siempre es un numero entero y la funcion va a devolver un arreglo*/
int* Crear_arr(int N){
    /*dirección de memoria para datos enteros*/
    int *v = (int*)malloc(sizeof(int)*N);
    for(int i=0;i<N;i++){
        v[i] = rand() %10;
    }
    return v;
}
/*el puntero ingresado es para dirección de un entero y la función devuelve la suma de num enteros*/
int suma_cubo(int* val, int N){
    int suma_c = 0;
    for(int i=0;i<N;i++){
        suma_c = suma_c + val[i]*val[i]*val[i];
    }
    return suma_c;
}
/*la cantidad de instrucciones son enteras y muchas*/
long int calc_num_inst(int N){
    return (9*N+7);
}
/* Los CPI son numeros decimales*/
/*los argumentos de entrada son enteros muy grandes*/
double calcular_CPI(long int cant_cicl,long int cant_inst){
    return (double)cant_cicl/((double)cant_inst);
}
/*Esta funcion tiene argumentos de entrada tipo double, por ende devolverá el mismo tipo de dato*/
double calcular_MIPS(double f, double CPI){
    return f/((CPI)*1e6);
}
/*Esta funcion va a imprimir*/
void Test(int N){
    int iterr = 100000;

    /*valores decimales*/
    double min_CPI = 1000, max_CPI = 0, min_MIPS = 1000, max_MIPS=0;
    double med_CPI, med_MIPS;
    double suma_CPI = 0, suma_MIPS = 0;

    /*dato obtenido anteriormente*/
    double f = 2800*1e6;
    
    for(int a=0;a<iterr;a++){
        /*Los valores son enteros muy grandes*/
        long int tic,toc,ciclos,num_inst;
        /*arreglo de enteros*/
        int* arr = Crear_arr(N);
        tic = __rdtsc();
        suma_cubo(arr,N);
        toc = __rdtsc();
        free(arr);
        ciclos = toc - tic;
        num_inst = calc_num_inst(N);

        /*calculando CPI*/
        double CPI = calcular_CPI(ciclos,num_inst);

        /*calc los MIPS*/
        double MIPS = calcular_MIPS(f,CPI);

        /*A continuación se obtendra la suma total de los CPI y MIPs con sus respectivos max y min*/
        /*Primero con el CPI*/

        suma_CPI = suma_CPI + CPI;

        if(min_CPI>CPI){
            min_CPI = CPI;
        }
        if(max_CPI<CPI){
            max_CPI = CPI;
        }

        /*Ahora con los MIPS*/
        suma_MIPS = suma_MIPS + MIPS;

        if(min_MIPS>MIPS){
            min_MIPS = MIPS;
        }
        if(max_MIPS<MIPS){
            max_MIPS = MIPS;
        }
    }
    /*Obteniendo la media de CPI y MIPS*/
    med_CPI = suma_CPI/iterr;
    med_MIPS = suma_MIPS/iterr;
    printf("Para el CPI\n");
    printf("El minímo valor es: %lf\nEl máximo valor es: %lf\nLa media es: %lf\n\n",min_CPI,max_CPI,med_CPI);
    printf("Para los MIPS\n");
    printf("El minímo valor es: %lf\nEl máximo valor es: %lf\nLa media es: %lf\n",min_MIPS,max_MIPS,med_MIPS);
    
}

int main(int argc, char const* argv[]){
    srand(time(NULL));
    int N;
    if(argc==1){
        N=16384;
        Test(N);
    }else{
        if(argc>2){
            printf("Está pasando más de la cantidad de argumentos requeridos. Solo se acepta como máximo uno.\n");
            return 0;
        }else{
            N = atoi(argv[1]);
            Test(N);
        }
    }
    return 0;
}

Overwriting main.c


In [165]:
! gcc main.c -o main


In [166]:
! ./main

Para el CPI
El minímo valor es: 0.599927
El máximo valor es: 18.059018
La media es: 0.618310

Para los MIPS
El minímo valor es: 155.047189
El máximo valor es: 4667.236371
La media es: 4556.680685


In [162]:
! ./main 100 200

Está pasando más de la cantidad de argumentos requeridos. Solo se acepta como máximo uno.


In [159]:
! ./main 32768

Para el CPI
El minímo valor es: 0.601304
El máximo valor es: 3.225692
La media es: 0.616432

Para los MIPS
El minímo valor es: 868.030669
El máximo valor es: 4656.545766
La media es: 4563.806062


In [163]:
! ./main 65536

Para el CPI
El minímo valor es: 0.602393
El máximo valor es: 2.163152
La media es: 0.620108

Para los MIPS
El minímo valor es: 1000.000000
El máximo valor es: 4648.129239
La media es: 4536.167344
